In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv("Walmart.csv")

In [3]:
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [4]:
df.columns

Index(['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6435 non-null   int64  
 1   Date          6435 non-null   object 
 2   Weekly_Sales  6435 non-null   float64
 3   Holiday_Flag  6435 non-null   int64  
 4   Temperature   6435 non-null   float64
 5   Fuel_Price    6435 non-null   float64
 6   CPI           6435 non-null   float64
 7   Unemployment  6435 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 402.3+ KB


In [6]:
df.isnull().sum()

Store           0
Date            0
Weekly_Sales    0
Holiday_Flag    0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
dtype: int64

In [7]:
df=df.drop(["Date"], axis=1)

In [8]:
y = df["Weekly_Sales"]
X = df.drop(["Weekly_Sales"], axis=1)

In [9]:
X.head()

,Store,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,0,42.31,2.572,211.096358,8.106
1,1,1,38.51,2.548,211.242170,8.106
2,1,0,39.93,2.514,211.289143,8.106
3,1,0,46.63,2.561,211.319643,8.106
4,1,0,46.50,2.625,211.350143,8.106


In [10]:
#Разделение
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.67, random_state=35)

In [11]:
#Случайный лес (группа бэггинга)
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)
prediction = rfr.predict(X_test)

In [12]:
params = { 
 'max_depth' : [10, 15, 20],
}

grid = GridSearchCV(estimator=RandomForestRegressor(), param_grid=params, cv = 5)
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [10, 15, 20]})

In [13]:
grid.best_score_, grid.best_params_

(0.9052476029800409, {'max_depth': 20})

In [14]:
#Группа бустинга
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
rfr = RandomForestRegressor(max_depth=15, random_state=42)
rfr.fit(X_train, y_train)
prediction = rfr.predict(X_test)
print(mean_absolute_error(y_test, prediction))
print(r2_score(y_test, prediction))

85885.83438571905
0.9163154857476018


In [15]:
#params = {
#    'n_estimators': [500, 800],
#    'max_depth': [5, 8],
#    'min_samples_split': [2, 5],
#    'learning_rate': [0.01, 0.1]
#}
#grid = GridSearchCV(GradientBoostingRegressor(), cv=5, n_jobs=-1)
#grid.fit(X_train, y_train)

In [16]:
#grid.best_score_, grid.best_params_

In [17]:
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train)
prediction = gbr.predict(X_test)
print(mean_absolute_error(y_test, prediction))
print(r2_score(y_test, prediction))

142411.5484111481
0.869218596157827


In [18]:
#Группа стекинга
reg1 = RandomForestRegressor(random_state=42)
reg2 = GradientBoostingRegressor(random_state=42)
reg3 = LinearRegression()
meta_learner = LinearRegression()
sr = StackingCVRegressor(regressors=[reg1, reg2, reg3], meta_regressor=meta_learner, random_state=42)
params = {
    'randomforestregressor__max_depth': [5, 8]
}

grid = GridSearchCV(estimator=sr, param_grid=params, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=StackingCVRegressor(meta_regressor=LinearRegression(),
                                           random_state=42,
                                           regressors=[RandomForestRegressor(random_state=42),
                                                       GradientBoostingRegressor(random_state=42),
                                                       LinearRegression()]),
             n_jobs=-1,
             param_grid={'randomforestregressor__max_depth': [5, 8]})

In [19]:
grid.best_score_, grid.best_params_

(0.9007658337229751, {'randomforestregressor__max_depth': 8})

In [20]:
sr = StackingCVRegressor(regressors=[reg1, reg2, reg3], meta_regressor=meta_learner, random_state=42)
sr.fit(X_train, y_train)
prediction = sr.predict(X_test)
print(mean_absolute_error(y_test, prediction))
print(r2_score(y_test, prediction))

88269.86515530065
0.9193865675100149


C:\Users\Артём Милевич\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
C:\Users\Артём Милевич\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
C:\Users\Артём Милевич\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
